In [3]:
import pandas as pd
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import os
dir_path = os.getcwd()


In [4]:
d_types = ['data', 'test']
data = dict()
for d_type in d_types:
        path = dir_path +"/"+"classification_data_HWK2/EMGaussian." + d_type 
        data[d_type] = pd.read_csv(path, sep=' ', header=None).values

In [5]:
print(data['data'][0])

[0.91029  0.016192]


In [8]:
a = np.ones((4,4))/4

In [11]:
# We use the parameters found in HW2 (We use our data)
pi = [0.20, 0.24, 0.25, 0.31]
mu = [[3.79, -3.64], [ 3.99, -3.64 ], [ -2.03, 4.16 ], [ -3.08, -3.56 ]]

sigma_0 = np.matrix([[ 0.87, 0.06 ], [ 0.06, 2.21]])
sigma_1 = np.matrix([[ 0.20, 0.22 ], [ 0.22, 10.40]])
sigma_2 = np.matrix([[ 2.92, 0.17 ], [ 0.17, 2.77]])
sigma_3 = np.matrix([[ 6,14, 5.94 ], [ 5.94, 6.07]])

sigma = [sigma_0, sigma_1, sigma_2, sigma_3]